In [ ]:
%config IPCompleter.greedy=True

In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import RMSprop
from keras_tuner import Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters

keras = tf.keras 

train_datagen = ImageDataGenerator(rescale=1./255) 
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/tmp/horse-or-human/', target_size=(150, 150), batch_size=32, class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('/tmp/validation-horse-or-human/', target_size=(150, 150), batch_size=32, class_mode='binary')

hp = HyperParameters()

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(hp.Choice('num_filter_layer_0', values=[16,64], default=16), (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(keras.layers.MaxPooling2D(2, 2))
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(keras.layers.Conv2D(hp.Choice(f'num_filter_layer_{i}', values=[16,64], default=16), (3, 3), activation='relu'))
        model.add(keras.layers.MaxPooling2D(2, 2))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hp.Int('hidden_units', 128, 512, step=32), activation='relu')) 
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['acc'])
    return model

tuner = Hyperband(build_model, objective='val_acc', max_epochs=15, directory='horse_human_params', project_name='mu_human_project')

tuner.search(train_generator, epochs=5, verbose=1, validation_data=validation_generator, validation_steps=8)

# history = model.fit(train_generator, epochs5, verbose=1, validation_data=validation_generator, validation_steps=8)

Trial 30 Complete [00h 22m 19s]
val_acc: 0.89453125

Best val_acc So Far: 0.92578125
Total elapsed time: 02h 51m 51s
INFO:tensorflow:Oracle triggered exit


In [2]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)
model = tuner.hypermodel.build(best_hps)
model.summary()

{'num_filter_layer_0': 64, 'num_conv_layers': 2, 'hidden_units': 320, 'num_filter_layer_1': 64, 'num_filter_layer_2': 64, 'tuner/epochs': 5, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 64)        36928     
_______________________________________________

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_width, img_height = 150, 150
img1 = image.load_img('/tmp/validation-horse-or-human/horses/horse1-000.png', target_size = (img_width, img_height))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis = 0)

img2 = image.load_img('/tmp/validation-horse-or-human/humans/valhuman01-00.png', target_size = (img_width, img_height))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis = 0)

print(model.predict(img1))

# print(model.predict(img2))

[[0.]]
